In [1]:
import os, sys
sys.path.append("motion_generation")
sys.path.append("rig_agnostic_encoding/functions")
sys.path.append("rig_agnostic_encoding/models")

from motion_generation.MoE import MoE
import motion_generation
from motion_generation.GRU import GRU
from motion_generation.LSTM import LSTM
from motion_generation.MotionGeneration import MotionGenerationModel
from motion_generation.MotionGeneration_v2 import MotionGenerationModel as MotionGenerationModel_v2
from motion_generation.MotionGeneration_v3 import MotionGenerationModel as MotionGenerationModel_v3
from motion_generation.MotionGenerationRNN import MotionGenerationModelRNN
from motion_generation.MotionGenerationBatch import MotionGenerationModelBatch
from rig_agnostic_encoding.models.MLP import MLP
from rig_agnostic_encoding.models.MLP_MIX import MLP_MIX
from rig_agnostic_encoding.models.VAE import VAE
from rig_agnostic_encoding.functions.DataProcessingFunctions import clean_checkpoints
from GlobalSettings import MODEL_PATH
import bz2

import torch
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import func as F
import _pickle as pickle
import json as js
import importlib

In [2]:
config = {
    "hidden_dim": 512,
    "k": 256,
    "lr": 1e-4,
    "batch_size": 32,
    "keep_prob": .2,
    "loss_fn":torch.nn.functional.mse_loss,
    "optimizer":torch.optim.Adam,
    "scheduler":torch.optim.lr_scheduler.StepLR,
    "scheduler_param": {"step_size":80, "gamma":.9},
    "basis_func":"gaussian",
    "n_centroid":128,
    "k_experts": 4,
    "gate_size": 128,
    "g_hidden_dim": 512,
    "num_layers": 4,
    "autoregress_prob":0,
    "autoregress_inc":.2,
    "autoregress_ep":50,
    "autoregress_max_prob":0,
    "cost_hidden_dim":128,
    "seq_len":15,
    "device":"cuda"
    }

In [3]:
MAX_FILES = -1
data_path = "/home/nuoc/Documents/MEX/data/Dataset_R1_One_1"
data_path2 = "/home/nuoc/Documents/MEX/data/data/Dataset_R2_One_1"
file_paths = []
file_paths2 = []
for dname, dirs, files in os.walk(data_path):
    for i, file in enumerate(files):
        file_paths.append(os.path.join(dname, file))
        if MAX_FILES > 0 and i >= MAX_FILES:
            break
for dname, dirs, files in os.walk(data_path2):
    for i, file in enumerate(files):
        file_paths2.append(os.path.join(dname, file))
        if MAX_FILES > 0 and i >= MAX_FILES:
            break


In [4]:
phase_features = ["phase_vec_l2"]
pose_features = ["pos", "rotMat2", "velocity"]
cost_features = ["posCost", "rotCost"]
features = phase_features + pose_features + cost_features
clips = []
feature_dims = {}

In [5]:
data = F.process_data_multithread(file_paths, features)
data2 = F.process_data_multithread(file_paths2, features, level=2)

feature_dims = data[0][1]
feature_dims2 = data2[0][1]

clips = [np.copy(d[0]) for d in data]
clips2 = [np.copy(d[0]) for d in data2]



2021-05-05 09:13:20,555	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-05 09:13:40,900	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


In [6]:
phase_dim = sum([feature_dims[feature] for feature in phase_features])
pose_dim = sum([feature_dims[feature] for feature in pose_features])
pose_dim2 = sum([feature_dims2[feature] for feature in pose_features])
cost_dim = sum([feature_dims[feature] for feature in cost_features])
print(phase_dim, " ", pose_dim, " ", cost_dim)
print(pose_dim2)

8   372   24
72


In [7]:
x_tensors = torch.stack([F.normaliseT(torch.from_numpy(clip)).float() for clip in clips])
y_tensors = torch.stack([torch.from_numpy(clip).float() for clip in clips])

x_tensors2 = torch.stack([F.normaliseT(torch.from_numpy(clip)).float() for clip in clips2])
y_tensors2 = torch.stack([torch.from_numpy(clip).float() for clip in clips2])

pose_data1 = x_tensors[:,  :,  phase_dim:phase_dim+pose_dim]
pose_data2 = x_tensors2[:, :, phase_dim:phase_dim+pose_dim2]
pose_data = torch.cat((pose_data1, pose_data2), dim=2)

dataset = TensorDataset(pose_data, pose_data)
dataset2 = TensorDataset(torch.Tensor(x_tensors2), torch.Tensor(y_tensors))

N = len(x_tensors)

train_ratio = int(.7*N)
val_ratio = int((N-train_ratio) / 2.0)
test_ratio = N - train_ratio - val_ratio
train_set, val_set, test_set = random_split(dataset, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set2, val_set2, test_set2 = random_split(dataset2, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
print(len(train_set), len(val_set), len(test_set))
print(len(train_set2), len(val_set2), len(test_set2))

168 36 36
168 36 36


In [9]:
dataset_name = "R1_MoGenData_full"
dataset_name2 = "R2_MoGenData_level=2"

In [12]:
data = {"data":[train_set, val_set, test_set],
        "feature_dims":feature_dims,
        "dims":[phase_dim, pose_dim, cost_dim]}
F.save(data, dataset_name, "/home/nuoc/Documents/MEX/data")

In [14]:
# obj = F.load("/home/nuoc/Documents/MEX/data/"+dataset_name+".pbz2")
# train_set, val_set, test_set = obj["data"]
# feature_dims = obj["feature_dims"]
# phase_dim, pose_dim, cost_dim= obj["dims"]

obj = F.load("/home/nuoc/Documents/MEX/data/"+dataset_name2+".pbz2")
train_set2, val_set2, test_set2 = obj["data"]
feature_dims2 = obj["feature_dims"]
phase_dim2, pose_dim2, cost_dim2= obj["dims"]

In [10]:
print(len(train_set), train_set[0][0].shape)
print(len(val_set), val_set[0][0].shape)
print(len(test_set), test_set[0][0].shape)
print(feature_dims)
print(feature_dims2)


168 torch.Size([300, 444])
36 torch.Size([300, 444])
36 torch.Size([300, 444])
{'phase_vec_l2': 8, 'pos': 93, 'rotMat2': 186, 'velocity': 93, 'posCost': 12, 'rotCost': 12}
{'phase_vec_l2': 8, 'pos': 18, 'rotMat2': 36, 'velocity': 18, 'posCost': 12, 'rotCost': 12}


In [18]:
model_name = "MLP_MIX_6-31"
MLPMIX = MLP_MIX(config=config, input_dims=[pose_dim, pose_dim2],
                 train_set=train_set, val_set=val_set, test_set=test_set)

In [19]:
MAX_EPOCHS = 200

checkpoint_callback = ModelCheckpoint(monitor="avg_val_loss", save_top_k=3)
earlystopping = EarlyStopping(monitor="avg_val_loss", patience=20)
logger=TensorBoardLogger(save_dir="logs/", name=model_name, version="0.1")

trainer = pl.Trainer(
    default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
    gpus=1, precision=16,
    callbacks=[earlystopping],
    min_epochs=20,
    logger=logger,
    max_epochs=MAX_EPOCHS,
    stochastic_weight_avg=True
)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [20]:
trainer.fit(MLPMIX)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | cluster_model | Sequential | 65.8 K
---------------------------------------------
65.8 K    Trainable params
0         Non-trainable params
65.8 K    Total params
0.263     Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f557c5d9640> for <torch.optim.swa_utils.SWALR object at 0x7f557d3130d0>
  warnings.warn(*args, **kwargs)


1

In [21]:
model_name = "MLP_MoE_R2_One_retarget_6-31"

feature_dims3 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim
    }

in_slice = [phase_dim, pose_dim2, cost_dim]
out_slice = [phase_dim, config["k"], cost_dim]

temp = MLP(config=config, dimensions=[pose_dim])
temp2 = MLP(config=config, dimensions=[pose_dim2])

pose_encoder = temp2
pose_encoder.encoder.load_state_dict(MLPMIX.active_models[1].encoder.state_dict())
pose_encoder.decoder = temp.decoder
pose_encoder.decoder.load_state_dict(MLPMIX.active_models[0].decoder.state_dict())

# pose_encoder = MLPMIX.active_models[1]
# pose_encoder.decoder = MLPMIX.active_models[1].decoder
middle_layer = MLPMIX.cluster_model
# cost_encoder = model2.cost_encoder

In [22]:
print(pose_encoder)

MLP(
  (encoder): Sequential(
    (fc0): Linear(in_features=72, out_features=512, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=512, out_features=512, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): Sequential(
    (fc0): Linear(in_features=256, out_features=512, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=512, out_features=512, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=512, out_features=372, bias=True)
  )
)


In [23]:
# generation_model = model2.generationModel
model = MotionGenerationModel_v2(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=feature_dims3,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=train_set2, val_set=val_set2, test_set=test_set2,
                                 name=model_name
                                   )
# model.generationModel = generation_model
# model.cost_encoder = cost_encoder
#
# model.cost_encoder.freeze()
# model.generationModel.freeze()
# model.middle_layer.requires_grad_(False)



In [24]:
MAX_EPOCHS = 300

checkpoint_callback = ModelCheckpoint(monitor="avg_val_loss", save_top_k=3)
earlystopping = EarlyStopping(monitor="avg_val_loss", patience=20)
logger=TensorBoardLogger(save_dir="logs/", name=model_name, version="0.1")

trainer = pl.Trainer(
    default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
    gpus=1, precision=16,
    callbacks=[earlystopping],
    min_epochs=20,
    logger=logger,
    max_epochs=MAX_EPOCHS,
    stochastic_weight_avg=True
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [25]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 1.0 M 
1 | middle_layer     | Sequential | 65.8 K
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 2.4 M 
------------------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.267    Total estimated model params size (MB)


1

In [26]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.08344020694494247, 'test_loss': 0.13288049399852753}
--------------------------------------------------------------------------------


[{'test_loss': 0.13288049399852753, 'ptl/test_loss': 0.08344020694494247}]

In [27]:
clean_checkpoints(path=os.path.join(MODEL_PATH,model_name))
# clean_checkpoints(path="/home/nuoc/Documents/MEX/models/version_0.2/MLP_MoE_R1_One_1_Full_v3_smooth_loss")

In [61]:
filename = "/home/nuoc/Documents/MEX/models/version_0.2/MLP_MoE_R1_One_1_Full_TUNE/0.06325527280569077.pbz2"
with bz2.BZ2File(filename, "rb") as f:
    obj = pickle.load(f)

pose_autoencoder = MLP.load_checkpoint(obj["pose_autoencoder_path"])
cost_encoder = MLP.load_checkpoint(obj["cost_encoder_path"])
generationModel = MoE.load_checkpoint(obj["motionGenerationModelPath"])

model = MotionGenerationModel_v2(config=obj["config"], feature_dims=obj["feature_dims"], Model=MoE,
                              input_slicers=obj["in_slices"], output_slicers=obj["out_slices"],
                              name=obj["name"])

model.in_slices = obj["in_slices"]
model.out_slices = obj["out_slices"]
model.pose_autoencoder = pose_autoencoder
model.cost_encoder = cost_encoder
model.generationModel = generationModel

In [64]:
model.test_set = test_set

[0, 62, 434, 458]
MLP(
  (encoder): Sequential(
    (fc0): Linear(in_features=372, out_features=512, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=512, out_features=512, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): Sequential(
    (fc0): Linear(in_features=256, out_features=512, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=512, out_features=512, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=512, out_features=372, bias=True)
  )
)
MLP(
  (encoder): Sequential(
    (fc0): Linear(in_features=24, out_features=128, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=128, out_features=128, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=128, out_features=128, bias=True)
  )
  (decoder): Sequent

In [29]:
model.autoregress_prob = 0
trainer.test(model)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.03257506340742111, 'test_loss': 0.07678437978029251}
--------------------------------------------------------------------------------


[{'test_loss': 0.07678437978029251, 'ptl/test_loss': 0.03257506340742111}]

In [30]:
model.autoregress_prob = 1
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.07573438435792923, 'test_loss': 0.10452431440353394}
--------------------------------------------------------------------------------


[{'test_loss': 0.10452431440353394, 'ptl/test_loss': 0.07573438435792923}]

In [28]:
# model.autoregress_prob = 0

n = 5
idx = np.random.randint(0, len(test_set), n)
original = []
generated = []
# pose_idx_upper = feature_dims2["phase_dim"] + feature_dims["pos"] + feature_dims["rotMat2"]
pose_idx_upper = model.in_slices[1] + feature_dims["pos"] + feature_dims["rotMat2"]
print(pose_idx_upper)


287


In [29]:
model.autoregress_prob = 1
with torch.no_grad():
    model.eval()
    model.cpu()
    # for i in range(1):
    # original =
    # x = x_tensors[idx]
    x = torch.stack([test_set2[i][0] for i in idx])
    y = torch.stack([test_set2[i][1] for i in idx])
    shape = x.shape
    # x = x.view(-1, shape[-1])

    x = x.view(-1, 15, shape[-1])
    # n = shape[1]
    # g_frames = [x_c.unsqueeze(1)]
    g_frames = []
    #
    for i in range(0, 15):
        x_c = x[:,i,:]
        out= model(x_c)
        x_c = torch.cat(out,dim=1).detach()
        g_frames.append(x_c.unsqueeze(1))

    # out = torch.cat(model(x), dim=1).view(shape)
            # x_c = torch.cat(out, dim=1)
    # g_frames.append(x_c.unsqueeze(1))
        # original.append(o_frames)
    # generated.append(torch.cat(g_frames, dim=1))
    # generated = out
    print(g_frames[0].shape, g_frames[1].shape, g_frames[2].shape)
    generated = torch.cat(g_frames, dim=1)
    generated = generated.view(y.shape)


torch.Size([100, 1, 404]) torch.Size([100, 1, 404]) torch.Size([100, 1, 404])


In [30]:
print(y.size())
# generated = generated.view(-1, 298, generated.shape[-1])
print(generated.size())

torch.Size([5, 300, 404])
torch.Size([5, 300, 404])


In [31]:
phase_dim = in_slice[0]
toPosDim = phase_dim+feature_dims["pos"]
toRotDim = toPosDim + feature_dims["rotMat2"]

gPos = generated[:, :, phase_dim:toPosDim]
gRot = generated[:, :, toPosDim:toRotDim]

oPos = y[:, :, phase_dim:toPosDim]
oRot = y[:, :, toPosDim:toRotDim]

print(gPos.shape, gRot.shape)
print(oPos.shape, oRot.shape)


torch.Size([5, 300, 93]) torch.Size([5, 300, 186])
torch.Size([5, 300, 93]) torch.Size([5, 300, 186])


In [32]:
n = 5
clip_length = gPos.shape[1]
gPos_r = gPos.reshape((n, clip_length, -1, 3))
gRot_r = gRot.reshape((n, clip_length, -1, 3, 2))
oPos_r = oPos.reshape((n, clip_length, -1, 3))
oRot_r = oRot.reshape((n, clip_length, -1, 3, 2))

print("Pos loss: ", torch.nn.functional.mse_loss(gPos_r, oPos_r))
print("Rot loss: ", torch.nn.functional.mse_loss(gRot_r, oRot_r))

Pos loss:  tensor(0.0090)
Rot loss:  tensor(0.0342)


In [33]:
template = js.load(open("/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json"))

In [34]:
def insert_pos(positions, rotations, name="Replay"):
    shape = positions.shape
    for c in range(shape[0]):
        for f in range(shape[1]):
            for j in range(shape[2]):
                template["frames"][f]["joints"][j]["position"]["x"] = positions[c,f,j,0].item()
                template["frames"][f]["joints"][j]["position"]["y"] = positions[c,f,j,1].item()
                template["frames"][f]["joints"][j]["position"]["z"] = positions[c,f,j,2].item()

                for r, cell in enumerate(["x", "y", "z"]):
                    for col, column in enumerate(["c0", "c1"]):
                        template["frames"][f]["joints"][j]["rotMat"][column][cell] = rotations[c,f,j,r, col].item()
        with open("{}_{}.json".format(name, c), "w") as f:
            js.dump(template, f)

insert_pos(oPos_r, oRot_r, "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/Replay_original_6-31_R2")
insert_pos(gPos_r, gRot_r, "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/Replay_generated_6-31_R2")
